In [1]:
import numpy as np
import pandas as pd
import sys
import random
import progressbar
import torch
import pickle
from mytree import *
from utils import *
from treeUtil import *
import tqdm
import argparse
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_
import functools
from sklearn.linear_model import LogisticRegression
import gensim
from gensim.models import KeyedVectors
import string
from stanfordcorenlp import StanfordCoreNLP
import nltk.tree
from ast import literal_eval
import pptree
import logging

ModuleNotFoundError: No module named 'mytree'

In [ ]:
from nltk.tokenize import word_tokenize
import re
import os
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from logger import Logger

In [ ]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from spacy.pipeline import merge_entities
nlp = en_core_web_sm.load()
nlp.add_pipe(merge_entities)

In [2]:
import copy
from sklearn.utils import shuffle
from sklearn import metrics

In [ ]:
# False if in-domain; True if general
proanti = True
w2vec = False
ner = True
blackout = False
balanced = True
undersample = False
non_trainable = True
economic = False

In [ ]:
def replace_entity(sent):
    doc = nlp(sent)
    for ent in doc.ents:
        if ent.label_=='PERSON':
            sent = sent.replace(ent.text,ent.label_)
    return sent

def blackout_entity(sent):
    doc = nlp(sent)
    for ent in doc.ents:
        if ent.label_=='PERSON':
            sent = sent.replace(ent.text,"")
    return sent

def read_into_string(filename):
    text_file = open(filename, 'r')
    lines = text_file.read().split('\n')
    if ner:
        lines = [replace_entity(sent) for sent in lines]
    if blackout:
        lines = [blackout_entity(sent) for sent in lines]
    text_file.close()
    return lines

In [ ]:
def process(file):
    sents = read_into_string(file)
    sents = [s for s in sents if len(s)>14 and len(s)<500]
    return sents

In [ ]:
if economic:
    policy = 'economic'
else:
    policy = 'tech'
policy

In [ ]:
# process csv to tsv
in_file = '/Users/navreetkaur/MTP/data/new/analysis/'+policy+'_mapping.csv'
out_file = '/Users/navreetkaur/MTP/data/new/analysis/'+policy+'_mapping_tsv.csv'
with open(in_file, 'r') as fi, open(out_file, 'w') as fo:
    for line in fi:
        line = line.replace(',', '\t', 2)
        fo.write(line)

In [ ]:
# converting statements to truecased
statement_file = '/Users/navreetkaur/MTP/data/new/analysis/'+policy+'_statements.txt'
df = pd.read_csv('/Users/navreetkaur/MTP/data/new/analysis/'+policy+'_mapping_tsv.csv', sep='\t', header='infer')
statements = list(df['By-Statement'])
with open(statement_file,'w') as f:
    for s in statements:
        f.write(s+'\n')
# -- run truecaser --
# 'python python PredictTruecaser.py -d distributions.obj'

In [ ]:
# make new tsv with processed(ner and shizz) lines
cased_statement_file = '/Users/navreetkaur/MTP/data/new/analysis/'+policy+'_statements.txt-cased'
final_cased_csv = '/Users/navreetkaur/MTP/data/new/analysis/final_'+policy+'_statements.csv'
cased_statements = read_into_string(cased_statement_file)
num_cannot_predict = []
new_cased_statements = []
for i, s in enumerate(cased_statements):
    if len(s)>14 and len(s)<500:
        new_cased_statements.append(cased_statements[i])
    else:
        num_cannot_predict.append(i)
num_cannot_predict, len(new_cased_statements)

In [ ]:
num_cannot_predict = num_cannot_predict[:-1]
num_cannot_predict

In [ ]:
df = df.drop(df.index[num_cannot_predict])

In [ ]:
cased_statements = new_cased_statements

In [ ]:
# cased_statements = cased_statements[:-1]
len(cased_statements), df.shape

In [ ]:
# with open(cased_statement_file, 'r') as f:
#     cased_statements = f.readlines()
# for i in range(len(cased_statements)):
#     cased_statements[i] = cased_statements[i].strip()
df['By-Statement'] = cased_statements
df.to_csv(final_cased_csv, sep='\t', index=False)

In [ ]:
df['tokens'] = [word_tokenize(re.sub(r'[^\w\s]|[\d]+',' ',sent)) for sent in df['By-Statement']]

In [ ]:
df.head()

In [ ]:
# make trees
# java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -annotators "tokenize,ssplit,pos,lemma,parse,sentiment" -port 9000 -timeout 90000
nlp = StanfordCoreNLP('http://localhost', port=9000,timeout=90000)

In [ ]:
def make_tree(text):
#     print(text)
    output = nlp.annotate(text, properties={
        'annotators': 'tokenize,ssplit,pos,depparse,parse',
        'outputFormat': 'json'
    })
    output = literal_eval(output)
    try:
        tree = str(output['sentences'][0]['parse'])
    except:
        print(output,text)
        return
    # print (tree)
    parse_string = ' '.join(str(tree).split())
    # print(parse_string)
    # print ("\n\n")
    tree = nltk.tree.Tree.fromstring(parse_string)
    tree.chomsky_normal_form()
    tree.collapse_unary(collapseRoot=True,collapsePOS=True)
    nt = convertNLTK_tree(tree)
    return nt

def printLabelTree(tree):
    def inorder(node,nnode):
        if node.isLeaf:
            newnode = pptree.Node('H',nnode)
            wnode = pptree.Node(node.word,newnode)
        elif nnode is not None:
            newnode = pptree.Node('H',nnode)
            inorder(node.left,newnode)
            inorder(node.right,newnode)
        elif node.isRoot():
            newnode = pptree.Node('H')
            inorder(node.left,newnode)
            inorder(node.right,newnode)
            return newnode
        return None
    pptree.print_tree(inorder(tree.root,None))

def create_trees_using_df(df):
    tree = []
    for tokens in list(df['tokens']):
        if len(tokens)==0:
            continue
        line = ' '.join(tokens)
        line += '\n'
        tree.append(make_tree(line))
    return tree

def printlabel(root,l):
    if root:
        l.append(root.label)
#         print(root.label)
        if root.left:
            l+=printlabel(root.left,[])
#             print(printlabel(root.left))
        if root.right:
            l+=printlabel(root.right,[])
#             print(printlabel(root.right))
    return l

In [ ]:
trees = create_trees_using_df(df)

In [ ]:
len(trees), df.shape

In [ ]:
df['tree'] = trees

In [ ]:
trees_file = '/Users/navreetkaur/MTP/data/new/analysis/'+policy+'_trees_df.pkl'
pickle.dump(df,open(trees_file,'wb'))

In [ ]:
dic = {0:'ProAnti', 1:'General', 2:'NER', 3:'Blackout', 4:'Balance', 5:'Undersample', 6:'Fixed'}
bool_list = [proanti, w2vec, ner, blackout, balanced, undersample, non_trainable]
corpus_path = '../data/new/'+policy
namecode = policy+'_w2v'
for index, bb in enumerate(bool_list):
    namecode += '_'
    if bb:
        namecode += '1'
    else:
        namecode += '0'
    namecode += dic[index]
namecode

In [ ]:
CUDA=False
def Var(v):
    if CUDA: return Variable(v.cuda())
    else: return Variable(v)
widgets = [progressbar.Percentage(), ' ', progressbar.Bar(), ' ', progressbar.ETA()]

In [ ]:
class RecursiveNN(nn.Module):
    def __init__(self, word_embeddings, vocab, embedSize=300, numClasses=2, beta = 0.3, use_weight = True, non_trainable = non_trainable):
        super(RecursiveNN, self).__init__()
#             if (w2vec):
#                 self.embedding = nn.Embedding(len(vocab), embedSize)
#                 self.embedding.load_state_dict({'weight': w2vec_weights})
        self.embedding = nn.Embedding.from_pretrained(word_embeddings)
        self.embedding.weight.requires_grad = True
        if non_trainable:
            self.embedding.weight.requires_grad = False
        else:
            self.embedding = nn.Embedding(len(vocab), embedSize)
        self.embedding = nn.Embedding(len(vocab), embedSize)
        self.W = nn.Linear(2*embedSize, embedSize, bias=True)
        self.nonLinear = torch.tanh
        self.projection = nn.Linear(embedSize, numClasses, bias=True)
        self.nodeProbList = []
        self.labelList = []
        self.loss = Var(torch.FloatTensor([0]))
        self.V = vocab
        self.beta = beta
        self.use_weight = use_weight
        self.total_rep = None #
        self.count_rep = 0 #
        self.numClasses = numClasses

    def traverse(self, node):
        if node.isLeaf:
            if node.getLeafWord() in self.V:  # check if right word is in vocabulary
                word = node.getLeafWord()
            else:  # otherwise use the unknown token
                word = 'UNK'
            # print(self.V[word],len(self.V),word,(torch.LongTensor([int(self.V[word])])))
            currentNode = (self.embedding(Var(torch.LongTensor([int(self.V[word])]))))
        else: currentNode = self.nonLinear(self.W(torch.cat((self.traverse(node.left),self.traverse(node.right)),1)))
        currentNode = currentNode/(torch.norm(currentNode))

        assert node.label!=None
        self.nodeProbList.append(self.projection(currentNode))
#         print(node.label)
#         print(torch.LongTensor([node.label]))
        self.labelList.append(torch.LongTensor([node.label]))
        loss_weight = 1-self.beta if node.annotated else self.beta
        self.loss += (loss_weight*F.cross_entropy(input=torch.cat([self.projection(currentNode)]),target=Var(torch.cat([torch.LongTensor([node.label])]))))

        #
        if not node.isRoot():
            if self.total_rep is None:
                self.total_rep = currentNode.data.clone()
            else:
                self.total_rep += currentNode.data.clone()
            self.count_rep += 1
        #
        return currentNode   
    
    
    
    def traverse_test(self, node):
        if node.isLeaf:
            if node.getLeafWord() in self.V:  # check if right word is in vocabulary
                word = node.getLeafWord()
            else:  # otherwise use the unknown token
                word = 'UNK'
            # print(self.V[word],len(self.V),word,(torch.LongTensor([int(self.V[word])])))
            currentNode = (self.embedding(Var(torch.LongTensor([int(self.V[word])]))))
        else: currentNode = self.nonLinear(self.W(torch.cat((self.traverse_test(node.left),self.traverse_test(node.right)),1)))
        currentNode = currentNode/(torch.norm(currentNode))

#         assert node.label!=None
        self.nodeProbList.append(self.projection(currentNode))
        loss_weight = 1-self.beta if node.annotated else self.beta

        if not node.isRoot():
            if self.total_rep is None:
                self.total_rep = currentNode.data.clone()
            else:
                self.total_rep += currentNode.data.clone()
            self.count_rep += 1
        #
        return currentNode
        

    def forward(self, x):
        self.nodeProbList = []
        self.labelList = []
        self.loss = Var(torch.FloatTensor([0]))
#         self.traverse(x)
        self.traverse_test(x)
#         self.labelList = Var(torch.cat(self.labelList))
        return torch.cat(self.nodeProbList)

    def getLoss(self, tree):
        nodes = self.forward(tree)
        predictions = nodes.max(dim=1)[1]
        loss = self.loss
        return predictions,loss
    
    
    def predict(self, trees):
        pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(trees)).start()
        preds = []
        for j, tree in enumerate(trees):
            nodes = self.forward(tree.root)
            predictions = nodes.max(dim=1)[1]
#                     print((predictions.cpu().data).numpy(),(predictions.cpu().data).numpy().shape)
#                     print((self.labelList.cpu().data).numpy(), (self.labelList.cpu().data).numpy().shape)
            preds.append(predictions)
            pbar.update(j)
        pbar.finish()
        return preds
    

    def getRep(self, tree):
        self.count_rep = 0
        self.total_rep = None
        self.nodeProbList = []
#         self.labelList = []
        self.loss = Var(torch.FloatTensor([0]))

        root_rep = self.traverse(tree)

        return (torch.cat((root_rep,self.total_rep/self.count_rep),1)).data.numpy().T.flatten()

    
#     def evaluate(self, trees):
#         pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(trees)).start()
#         n = nAll = correctRoot = correctAll = 0.0
#         tp = [1e-2]*self.numClasses
#         fp = [1e-2]*self.numClasses
#         fn = [1e-2]*self.numClasses
#         f1 = [0.]*self.numClasses
#         for j, tree in enumerate(trees):
#             predictions,_ = self.getLoss(tree.root)
# #                     print((predictions.cpu().data).numpy(),(predictions.cpu().data).numpy().shape)
# #                     print((self.labelList.cpu().data).numpy(), (self.labelList.cpu().data).numpy().shape)
#             correct = ((predictions.cpu().data).numpy()==(self.labelList.cpu().data).numpy())
# #                     print(correct)
#             correctAll += correct.sum()
#             nAll += np.shape(correct.squeeze())[0] if np.size(correct)!=1 else 1 
#             correctRoot += correct.squeeze()[-1] if np.size(correct)!=1 else correct[-1]
# #                     print(correct.squeeze()[-1] if np.size(correct)!=1 else correct[-1])
# #                     print('actual: {}'.format(tree.root.label))
#             for i in range(self.numClasses):
#                 size = np.size((predictions.cpu().data).numpy())
#                 if size!=1:
#                     pred = (predictions.cpu().data).numpy().squeeze()[-1]
#                     actual = (self.labelList.cpu().data).numpy().squeeze()[-1]
#                 else:
#                     pred = (predictions.cpu().data).numpy()[-1]
#                     actual = (self.labelList.cpu().data).numpy()[-1]
#                 if pred==i and actual==i:
#                     tp[i]+=1
#                 elif pred==i and actual!=i:
#                     fn[i]+=1
#                 elif pred==i and actual!=i:
#                     fp[i]+=1
#             n += 1
#             pbar.update(j)
# #             print(tp,fp,fn)
#         for i in range(self.numClasses):
#             p =(1.0*tp[i]/(tp[i]+fp[i]))
#             r =(1.0*tp[i]/(tp[i]+fn[i]))
#             f1[i] = (2*p*r)/(p+r)
#         pbar.finish()
#         return correctRoot / n, correctAll/nAll, f1

    def eval_sent_lvl(self,trees,clf):
        pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(trees)).start()
        n = nAll = correctRoot = correctAll = 0.0
        X_predict = []
        Y_gold = []
        for j, tree in enumerate(trees):
            tree_rep = model.getRep(tree.root)
            X_predict.append(tree_rep)
            Y_gold.append(tree.root.label)
        acc = clf.score(np.array(X_predict),np.array(Y_gold))
        return acc

In [ ]:
namecode

In [ ]:
df = pickle.load(open('/Users/navreetkaur/MTP/data/new/analysis/'+policy+'_trees_df.pkl', 'rb'))
test_trees = df['tree']
model = pickle.load(open("/Users/navreetkaur/MTP/tech_classifier/models/"+namecode+'.pkl','rb'))
preds = model.predict(test_trees)
preds = [p.numpy()[-1] for p in preds]

labels = []
i=0
for p in preds:
    if p==0:
        labels.append('PRO')
    else:
        labels.append('ANTI')
        i+=1

df['label'] = labels

In [ ]:
pickle.dump(df,open('/Users/navreetkaur/MTP/data/new/analysis/pred_'+policy+'.pkl','wb'))

In [ ]:
# for i, name in enumerate(names):
#     df_pred = predictions[i]
#     print(name, "PRO: ", len(df_pred[df_pred['Label']=='PRO']), "ANTI: ", len(df_pred[df_pred['Label']=='ANTI']))

## Analysis

In [ ]:
df_econ = pickle.load(open('/Users/navreetkaur/MTP/data/new/analysis/pred_economic.pkl', 'rb'))
df_tech = pickle.load(open('/Users/navreetkaur/MTP/data/new/analysis/pred_tech.pkl', 'rb'))

In [9]:
df_econ = pd.read_csv('/Users/ankursharma/Desktop/MTP-org/gem2data/econ_mapping_label.csv', delimiter='#;;#')

/Users/ankursharma/anaconda3/envs/cv2/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [10]:
df_econ.head()

,Entity,Source,label,By-Statement
0,Ajay Kumar,NIE,Anti,alleging that modi has not fulfilled any of th...
1,Ajay Kumar,TOI,Pro,"additional district magistrate ajaykumar said,..."
2,Ajit Pawar,TOI,Pro,asserting that the interests of farmers will b...
3,Ajit Pawar,IE,Anti,", ajitpawar had said in an insensitive comment..."
4,Ajit Pawar,IE,Pro,b'amid clamour for his resignation over the co...


In [ ]:
df_tech.head()

## Entity Wise

In [11]:
df_econ.groupby(['Entity','label']).agg(['count'])

Source By-Statement
                       count        count
Entity          label                    
Ajay Kumar      Anti       1            1
                Pro        1            1
Ajit Pawar      Anti       1            1
                Pro        2            2
Akhilesh Yadav  Anti       8            8
...                      ...          ...
Vasundhara Raje Anti       1            1
                Pro        6            6
Vijay Rupani    Pro        4            4
Yogi Adityanath Anti      10           10
                Pro       25           25

[127 rows x 2 columns]

In [13]:
# df_econ_entity = pd.DataFrame({'count': df_econ.groupby(['Entity','label']).agg(['count']).size()}).reset_index()
df_econ_entity = df_econ.groupby(['Entity','label']).count()
df_econ_entity = df_econ_entity.drop(columns=['Source'])
df_econ_entity.to_csv('/Users/ankursharma/Desktop/MTP-org/gem2data/analysis/economic_entity_rcnn.csv')
df_econ_entity

By-Statement
Entity          label              
Ajay Kumar      Anti              1
                Pro               1
Ajit Pawar      Anti              1
                Pro               2
Akhilesh Yadav  Anti              8
...                             ...
Vasundhara Raje Anti              1
                Pro               6
Vijay Rupani    Pro               4
Yogi Adityanath Anti             10
                Pro              25

[127 rows x 1 columns]

In [14]:
df_tech.groupby(['Entity', 'label']).agg(['count'])

NameError: name 'df_tech' is not defined

In [ ]:
df_tech_entity = df_tech.groupby(['Entity','label']).count()
df_tech_entity = df_tech_entity.drop(columns=['Source','tokens','tree'])
df_tech_entity.to_csv('/Users/navreetkaur/MTP/data/new/analysis/tech_entity.csv')
df_tech_entity

## Media Source wise

In [15]:
df_econ.groupby(['Source','label']).agg(['count'])

Entity By-Statement
                 count        count
Source    label                    
DecH      Anti     212          212
          Pro      591          591
HT        Anti      64           64
          Pro      154          154
IE        Anti     169          169
          Pro      494          494
NIE       Anti     218          218
          Pro      463          463
TOI       Anti      71           71
          Pro      152          152
Telegraph Anti      58           58
          Pro      135          135

In [18]:
df_econ_media = df_econ.groupby(['Source','label']).count()
df_econ_media = df_econ_media.drop(columns=['Entity'])
df_econ_media.to_csv('/Users/ankursharma/Desktop/MTP-org/gem2data/analysis/economic_media_rcnn.csv')
df_econ_media

By-Statement
Source    label              
DecH      Anti            212
          Pro             591
HT        Anti             64
          Pro             154
IE        Anti            169
          Pro             494
NIE       Anti            218
          Pro             463
TOI       Anti             71
          Pro             152
Telegraph Anti             58
          Pro             135

In [ ]:
df_tech.groupby(['Source','label']).agg(['count'])

In [ ]:
df_tech_media = df_tech.groupby(['Source','label']).count()
df_tech_media = df_tech_media.drop(columns=['Entity','tokens','tree'])
df_tech_media.to_csv('/Users/navreetkaur/MTP/data/new/analysis/tech_media.csv')

In [ ]:
df_tech_media

In [19]:
df_econ

,Entity,Source,label,By-Statement
0,Ajay Kumar,NIE,Anti,alleging that modi has not fulfilled any of th...
1,Ajay Kumar,TOI,Pro,"additional district magistrate ajaykumar said,..."
2,Ajit Pawar,TOI,Pro,asserting that the interests of farmers will b...
3,Ajit Pawar,IE,Anti,", ajitpawar had said in an insensitive comment..."
4,Ajit Pawar,IE,Pro,b'amid clamour for his resignation over the co...
...,...,...,...,...
2776,Yogi Adityanath,IE,Anti,after uttar pradesh chief minister yogiadityan...
2777,Yogi Adityanath,IE,Pro,"at the function, yogiadityanath said, the 86 l..."
2778,Yogi Adityanath,IE,Pro,"as per the constitutional provisions, the sche..."
2779,Yogi Adityanath,IE,Pro,"as per the constitutional provisions, the sche..."
